#### 116개 파일 > 행단위로 통합하기

In [ ]:
"""
<지금까지 처리한 순서>
 1. 파일 읽어들이기
 2. [기준년월] 컬럼 추가
 3. [국적] 컬럼에서 [대륙] 이름 제거
 4. "인덱스" 재정의
 5. [대륙] 컬럼 추가
 6. [관광객비율(%)] 컬럼 추가
 7. [전체관광객비율(%)] 컬럼 추가
"""

In [7]:
import pandas as pd

# 깨끗한 데이터프레임 정의(통합본)
df = pd.DataFrame()

# 116개의 파일을 읽어 들일 수 있도록 파일명 생성하기
# -파일명의 패턴을 반복문을 이용해서 사용(년도,월)
for y in range(2010, 2020, 1) :
    for m in range(1, 13, 1) :
        file_path = f"./files2/kto_{y}{str(m).zfill(2)}.xlsx"
        #print(file_path)
        #예외처리 try~except
        # - 외부 영역에서 어떤 오류가 발생할지 모르는 경우에 예외처리 한다.

        try :
            kto_201901 = pd.read_excel(file_path,
                                  header=1,
                                  skipfooter=4,
                                  usecols="A:G")
            # 기준년월 추가
            kto_201901["기준년월"] = f"{y}-{str(m).zfill(2)}"

            # 국적컬럼에서 대륙이름 제거
            c = kto_201901["국적"].isin(['아시아주', '미주','구주', '대양주','아프리카주', '기타대륙', '교포소계'])
            kto_201901_country = kto_201901[c==False]

            # 인덱스번호 재정의
            kto_201901_country_newIndex = kto_201901_country.reset_index(drop=True)

            #대륙컬럼 추가하기
            continents = ['아시아']*25 + ['아메리카']*5 + ['유럽']*23 + ['오세아니아']*3 \
                            + ['아프리카']*2 + ['기타대륙'] + ['교포']
            kto_201901_country_newIndex['대륙'] = continents

            #관광객비율% 컬럼 추가
            kto_201901_country_newIndex["관광객 비율(%)"] \
                = round(kto_201901_country_newIndex["관광"] / kto_201901_country_newIndex["계"] *100, 3)

            #전체관광객비율% 컬럼추가
            t_sum = kto_201901_country_newIndex["관광"].sum()

            kto_201901_country_newIndex["전체관광객비율(%)"] \
                = round(kto_201901_country_newIndex["관광"] / t_sum *100,3)

            # 원본 df 데이터프레임에 위 데이터프레임 행단위 추가하기
            df = pd.concat([df, kto_201901_country_newIndex], ignore_index=True)
            
        except :
            # 오류가 발생할 경우 처리되는 영역
            break
    


In [8]:
# 통합본 파일로 저장하기
save_path = "./files2_tot/kto_total.xlsx"

df.to_excel(save_path, index=False)

#### OS 라이브러리를 이용하여 파일 읽기

In [9]:
import os
import pandas as pd

In [ ]:
### 현재 작업하는 주피터 파일이 위치한 폴더 위치 조회
# - 절대경로로 알려줌
root = os.getcwd()
# print(root)

### root 경로 하위에 있는 폴더 및 파일 이름을 모두 조회하기
# - 리스트 타입으로 반환받기
# - 내가 사용할 파일 또는 폴더의 인덱스 번호를 사용하여 추출하기
dir_nm = os.listdir(root)[2]
# print(dir_nm)

### 내가 사용할 폴더 하위의 모든 파일들 조회하기
# - 리스트 타입으로 반환받기
file_list = os.listdir(dir_nm)

### 깨끗한 데이터프레임 정의하기
df = pd.DataFrame()

### file_list 만큼 반복하기
for file_nm in file_list :
    ### 파일 경로 정의하기
    # file_path = f"{root}/{dir_nm}/{file_nm}"
    file_path = f"./{dir_nm}/{file_nm}"
    # print(file_path)

    try :
        kto_201901 = pd.read_excel(file_path,
                                   header=1,
                                   skipfooter=4,
                                   usecols="A:G")

        # 2. [기준년월] 컬럼 추가
        kto_201901["기준년월"] = f"{file_nm[4:8]}-{file_nm[8:10]}"
        
        # 3. [국적] 컬럼에서 [대륙] 이름 제거
        c = kto_201901["국적"].isin(['아시아주', '미주', '구주', '대양주', 
                                     '아프리카주', '기타대륙', '교포소계'])
        kto_201901_country = kto_201901[c == False]
        
        # 4. "인덱스" 재정의
        kto_201901_country_newindex = kto_201901_country.reset_index(drop=True)
        
        # 5. [대륙] 컬럼 추가
        continents = ["아시아"]*25 + ["아메리카"]*5 + ["유럽"]*23 + ["오세아니아"]*3 + \
                        ["아프리카"]*2 + ["기타대륙"] + ["교포"]
        kto_201901_country_newindex["대륙"] = continents
        
        # 6. [관광객비율(%)] 컬럼 추가
        kto_201901_country_newindex["관광객비율(%)"] = \
                round(kto_201901_country_newindex["관광"] / \
                      kto_201901_country_newindex["계"] * 100, 3)
        
        # 7. [전체관광객비율(%)] 컬럼 추가
        t_sum = kto_201901_country_newindex["관광"].sum()
        kto_201901_country_newindex["전체관광객비율(%)"] = \
                    round(kto_201901_country_newindex["관광"] / t_sum * 100, 3)

        # 8. 원본 df 데이터프레임에 위 데이터프레임 행단위 추가하기
        df = pd.concat([df, kto_201901_country_newindex], ignore_index=True)
        
    except : 
        ### 오류가 발생할 경우 처리되는 영역
        break

In [16]:
### 통합본 파일로 저장하기
save_path = "./files2_tot/kto_total2.xlsx"
df.to_excel(save_path, index=False)